<a href="https://colab.research.google.com/github/Yash-Jumde/Colab-Notebooks/blob/main/LinearRegression_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'openintro-possum:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1534513%2F2532158%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240305%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240305T205048Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2aef5a7052327aeed6feeccdb248e69514570e1151b3e085801c2e5e2330c071889767c97d9f3ddabedf5cd586efaf7276d881506722648856294f2d8f934e267de07c1e96e3232c6f5254c3445cc2cdc4283cf25e540ab9ed873b760995801f440a8656d90924ebcf098574399906f1a754d651f49cc4e6bfd827d2125602cb80bfab59186d2017c4fac3ac3a9cf427c03bf96c8e9cfb2b125900389aeb8ea971a85310a9d55a4801f2f793247c9b6af11cb597e8a16e10c4689b179bf21a083db25c2862657d85e996fb680972379f78272d77b5d4e403631ae3ca47a0da29e35e1e4f713f4743ce3ce97f66ffd5fb60e19eb43703baaafef16fdd220587ac'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 2203 bytes downloaded
Downloaded and uncompressed: openintro-possum
Data source import complete.


In [ ]:
import tensorflow as tf
import pandas as pd

In [ ]:
df = pd.read_csv('/kaggle/input/openintro-possum/possum.csv')

In [ ]:
df.dropna(inplace=True)

In [ ]:
data_hot_encoded = pd.get_dummies(df)
data_hot_encoded

,case,site,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly,Pop_Vic,Pop_other,sex_f,sex_m
0,1,1,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0,1,0,0,1
1,2,1,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0,1,0,1,0
2,3,1,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0,1,0,1,0
3,4,1,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0,1,0,1,0
4,5,1,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,100,7,1.0,89.5,56.0,81.5,36.5,66.0,46.8,14.8,23.0,27.0,0,1,0,1
100,101,7,1.0,88.6,54.7,82.5,39.0,64.4,48.0,14.0,25.0,33.0,0,1,0,1
101,102,7,6.0,92.4,55.0,89.0,38.0,63.5,45.4,13.0,25.0,30.0,0,1,1,0
102,103,7,4.0,91.5,55.2,82.5,36.5,62.9,45.9,15.4,25.0,29.0,0,1,0,1


In [ ]:
data_hot_encoded.drop(['case', 'site'], axis=1,inplace=True)

In [ ]:
data_hot_encoded

,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly,Pop_Vic,Pop_other,sex_f,sex_m
0,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0,1,0,0,1
1,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0,1,0,1,0
2,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0,1,0,1,0
3,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0,1,0,1,0
4,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,1.0,89.5,56.0,81.5,36.5,66.0,46.8,14.8,23.0,27.0,0,1,0,1
100,1.0,88.6,54.7,82.5,39.0,64.4,48.0,14.0,25.0,33.0,0,1,0,1
101,6.0,92.4,55.0,89.0,38.0,63.5,45.4,13.0,25.0,30.0,0,1,1,0
102,4.0,91.5,55.2,82.5,36.5,62.9,45.9,15.4,25.0,29.0,0,1,0,1


In [ ]:
X = data_hot_encoded.drop('age', axis=1)
y = data_hot_encoded['age']

In [ ]:
X, y

(     hdlngth  skullw  totlngth  taill  footlgth  earconch   eye  chest  belly  \
 0       94.1    60.4      89.0   36.0      74.5      54.5  15.2   28.0   36.0   
 1       92.5    57.6      91.5   36.5      72.5      51.2  16.0   28.5   33.0   
 2       94.0    60.0      95.5   39.0      75.4      51.9  15.5   30.0   34.0   
 3       93.2    57.1      92.0   38.0      76.1      52.2  15.2   28.0   34.0   
 4       91.5    56.3      85.5   36.0      71.0      53.2  15.1   28.5   33.0   
 ..       ...     ...       ...    ...       ...       ...   ...    ...    ...   
 99      89.5    56.0      81.5   36.5      66.0      46.8  14.8   23.0   27.0   
 100     88.6    54.7      82.5   39.0      64.4      48.0  14.0   25.0   33.0   
 101     92.4    55.0      89.0   38.0      63.5      45.4  13.0   25.0   30.0   
 102     91.5    55.2      82.5   36.5      62.9      45.9  15.4   25.0   29.0   
 103     93.6    59.9      89.0   40.0      67.6      46.0  14.8   28.5   33.5   
 
      Pop_Vic 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
len(X_train), len(X_test)

(80, 21)

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler

In [ ]:
ct = make_column_transformer(
    (StandardScaler(), ['hdlngth', 'skullw', 'totlngth', 'taill', 'footlgth', 'earconch', 'eye', 'chest', 'belly'])
)

ct.fit(X_train)

X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)

In [ ]:
X_train_normal[0].dtype, y_train

(dtype('float64'),
 94    4.0
 74    6.0
 56    4.0
 51    6.0
 48    4.0
      ... 
 92    3.0
 28    3.0
 67    5.0
 15    4.0
 9     6.0
 Name: age, Length: 80, dtype: float64)

In [ ]:
tf.random.set_seed(10)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

model.compile(loss=tf.keras.losses.mae,
             optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.fit(X_train_normal, y_train, epochs=100)

Epoch 1/100
3/3 [==============================] - 2s 14ms/step - loss: 3.6755
Epoch 2/100
3/3 [==============================] - 0s 13ms/step - loss: 3.5194
Epoch 3/100
3/3 [==============================] - 0s 12ms/step - loss: 3.3749
Epoch 4/100
3/3 [==============================] - 0s 11ms/step - loss: 3.2448
Epoch 5/100
3/3 [==============================] - 0s 9ms/step - loss: 3.1137
Epoch 6/100
3/3 [==============================] - 0s 9ms/step - loss: 2.9815
Epoch 7/100
3/3 [==============================] - 0s 10ms/step - loss: 2.8540
Epoch 8/100
3/3 [==============================] - 0s 11ms/step - loss: 2.7154
Epoch 9/100
3/3 [==============================] - 0s 13ms/step - loss: 2.5971
Epoch 10/100
3/3 [==============================] - 0s 9ms/step - loss: 2.4669
Epoch 11/100
3/3 [==============================] - 0s 9ms/step - loss: 2.3484
Epoch 12/100
3/3 [==============================] - 0s 9ms/step - loss: 2.2210
Epoch 13/100
3/3 [==============================] - 0s

In [ ]:
model.predict(X_test_normal)

1/1 [==============================] - 0s 73ms/step


array([[3.6251495],
       [4.931413 ],
       [4.463715 ],
       [1.9361055],
       [1.1924164],
       [4.332962 ],
       [3.6446433],
       [4.0943913],
       [4.8511834],
       [3.5951643],
       [1.8498094],
       [3.9049103],
       [3.9881713],
       [2.560135 ],
       [5.036209 ],
       [4.8835244],
       [2.1261446],
       [5.2106495],
       [1.4068029],
       [3.950375 ],
       [3.7799387]], dtype=float32)

In [ ]:
y_test

82    3.0
66    4.0
88    6.0
39    3.0
38    1.0
3     6.0
19    4.0
69    7.0
93    7.0
14    5.0
46    2.0
2     6.0
1     6.0
44    3.0
64    5.0
47    5.0
26    2.0
98    3.0
78    1.0
85    3.0
84    3.0
Name: age, dtype: float64

In [ ]:
from sklearn.linear_model import LinearRegression

regression_model = LinearRegression()
regression_model.fit(X_train_normal, y_train)

LinearRegression()

In [ ]:
y_pred = regression_model.predict(X_test_normal)
y_pred

array([2.40847493, 3.68746208, 3.5891019 , 2.59415951, 1.81308261,
       3.81121274, 4.45789516, 2.81607662, 3.70844676, 4.46622787,
       3.29130112, 4.25257045, 4.126137  , 3.53739586, 3.32939978,
       4.69658281, 2.40024753, 3.50596737, 2.06311747, 2.86395807,
       3.00525404])